In [1]:
import pandas as pd
import numpy as np
import os

In [6]:
import sqlalchemy as db
from sqlalchemy import create_engine, MetaData, inspect

# Data
connecting to the text message database using sql alchemy.

**** thoughts -  will I have to store this data in a separte table/db after cleaning? seems like an extra step if its alreayd all in one place.unsure if when making public I will be able to query my personal data. Also - longevity. 
do these texts die?

In [7]:
engine = db.create_engine('sqlite:////Users/michaelharnett/Library/Messages/chat.db')

In [8]:
engine

Engine(sqlite:////Users/michaelharnett/Library/Messages/chat.db)

In [9]:
connection = engine.connect()

In [10]:
metadata = db.MetaData()

In [11]:
metadata

MetaData()

In [12]:
db.tables()

AttributeError: module 'sqlalchemy' has no attribute 'tables'

In [13]:
metadata.tables.keys()

dict_keys([])

In [14]:
engine.table_names()

/var/folders/xl/ftndz_9j3hn5ybffb6t5q42m0000gn/T/ipykernel_2699/4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['_SqliteDatabaseProperties',
 'attachment',
 'chat',
 'chat_handle_join',
 'chat_message_join',
 'deleted_messages',
 'handle',
 'kvtable',
 'message',
 'message_attachment_join',
 'message_processing_task',
 'sqlite_sequence',
 'sqlite_stat1',
 'sync_deleted_attachments',
 'sync_deleted_chats',
 'sync_deleted_messages']

In [15]:
insp = inspect(engine)
insp

In [16]:
db.Table()

AttributeError: 'Table' object has no attribute 'name'

In [17]:
# pulling in query I found online/worked on in DB Browser. For somereason
#I could not get it to work unless all on one line, so it is one long gross line.

orig_df = pd.read_sql("select distinct m.rowid ,m.is_from_me IsFromMe ,case when m.is_from_me = 1 then m.account else h.id end as FromPhoneNumber ,datetime((m.date / 1000000000) + 978307200, 'unixepoch', 'localtime') as TextDate, m.text MessageText,c.display_name RoomName from message as m left join handle as h on m.handle_id = h.rowid left join chat as c on m.cache_roomnames = c.room_name left join chat_handle_join as ch on c.rowid = ch.chat_id left join handle as h2 on ch.handle_id = h2.rowid where RoomName LIKE 'Wordle Fam%' order by m.date desc;", engine)

# EDA BABY


In [18]:
df = orig_df[['FromPhoneNumber','TextDate','MessageText']] # taking only the fields I need

In [19]:


df = df.rename(columns = {'FromPhoneNumber':'player', 'TextDate':'date', 'MessageText':'message'})

In [20]:
df

,player,date,message
0,+15514868670,2022-03-26 09:10:04,Wordle 280 4/6\n\n⬛️🟨⬛️⬛️⬛️\n⬛️⬛️🟨⬛️⬛️\n⬛️🟨⬛️🟨...
1,None,2022-03-26 02:07:54,None
2,+12017253072,2022-03-26 00:50:25,😊
3,+12014523662,2022-03-26 00:50:01,Wordle 280 5/6\n\n⬛⬛🟨⬛⬛\n⬛⬛🟩⬛🟨\n🟩⬛🟩⬛⬛\n🟩⬛🟩⬛🟩\n...
4,+12017253072,2022-03-26 00:42:45,Wordle 280 3/6\n\n⬜🟩🟩⬜⬜\n🟩🟩🟩⬜⬜\n🟩🟩🟩🟩🟩
...,...,...,...
585,E:michaelcharnett@gmail.com,2022-02-03 19:12:58,I got wordle in 2 today 😏
586,E:michaelcharnett@gmail.com,2021-07-20 13:36:50,￼
587,+12017253072,2021-07-20 13:22:56,Thank you😊
588,+12014523662,2021-07-20 13:22:22,Yes it’s my email and password is Lucky95


In [77]:
# Renaming changing the Id from phone number to our names

df.loc[df.player == '+12014523662', 'player'] = 'Danielle'
df.loc[df.player == '+12017253072', 'player'] = 'Dad'
df.loc[df.player == '+15514868670', 'player'] = 'Chris'
df.loc[df.player == 'E:michaelcharnett@gmail.com', 'player'] = 'Michael'

In [22]:
df.message.str.contains('Wordle')

0       True
1       None
2      False
3       True
4       True
       ...  
585    False
586    False
587    False
588    False
589    False
Name: message, Length: 590, dtype: object

In [23]:
df.player.value_counts()

Michael     173
Danielle    170
Dad         142
Chris       101
Name: player, dtype: int64

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   player   586 non-null    object
 1   date     590 non-null    object
 2   message  586 non-null    object
dtypes: object(3)
memory usage: 14.0+ KB


#### trying to figure the best way to filter the messages
only want this to look at the messages that we send from a completed game. they all start Wordle ### with the game number. Im glad that number is included, it will be easier to join on that!

In [83]:
df.message.astype('str')[3][:5]=='Wordl'

True

In [85]:
df.message[3][:5]=='Wordl'

True

In [94]:
df[df.player == 'None']

,player,date,message


In [98]:
df = df.dropna()

In [26]:
df[df.message.str.startswith("Wor")]

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [109]:
df[df.message.str.contains('Wordle')]

,player,date,message
2,Danielle,2022-03-26 00:50:01,Wordle 280 5/6\n\n⬛⬛🟨⬛⬛\n⬛⬛🟩⬛🟨\n🟩⬛🟩⬛⬛\n🟩⬛🟩⬛🟩\n...
3,Dad,2022-03-26 00:42:45,Wordle 280 3/6\n\n⬜🟩🟩⬜⬜\n🟩🟩🟩⬜⬜\n🟩🟩🟩🟩🟩
8,Michael,2022-03-25 19:05:29,Wordle 279 4/6\n\n🟨🟩⬛⬛⬛\n⬛🟩⬛🟨⬛\n🟨🟩🟩⬛🟨\n🟩🟩🟩🟩🟩\n...
9,Dad,2022-03-25 01:05:31,Wordle 279 3/6\n\n⬜⬜⬜⬜⬜\n🟨🟨⬜🟨⬜\n🟩🟩🟩🟩🟩
10,Danielle,2022-03-25 00:20:43,Wordle 279 4/6\n\n🟨🟩⬛⬛⬛\n⬛🟩🟨⬛⬛\n⬛🟩⬛🟨🟨\n🟩🟩🟩🟩🟩
...,...,...,...
574,Michael,2022-02-05 10:31:55,Wordle 231 4/6\n\n🟨⬛🟨⬛⬛\n🟩🟩⬛⬛⬛\n🟩🟩🟩⬛🟨\n🟩🟩🟩🟩🟩
576,Dad,2022-02-05 01:08:40,￼Wordle 231 3/6\n\n3 for me \n\n⬜🟨⬜🟨⬜\n🟨🟩🟩🟨🟩\n...
577,Michael,2022-02-04 08:41:31,Wordle 230 5/6\n\n🟨🟨⬛⬛⬛\n🟨🟩⬛⬛🟨\n⬛🟩🟩⬛🟨\n🟩🟩🟩🟩⬛\n...
578,Danielle,2022-02-04 08:39:54,Wordle 230 4/6\n\n⬛🟨⬛⬛🟨\n⬛🟨🟨🟨⬛\n🟩🟨🟨🟨🟩\n🟩🟩🟩🟩🟩


In [101]:
#df[df.message.str.contains('🟩')] # interesting i'm surprised this works!

## 

#df[df.message.str.contains('Wordle') & df.message.str.contains('🟩')] # really should use this, but unsure if the green chracter will trhow error in production

,player,date,message
2,Danielle,2022-03-26 00:50:01,Wordle 280 5/6\n\n⬛⬛🟨⬛⬛\n⬛⬛🟩⬛🟨\n🟩⬛🟩⬛⬛\n🟩⬛🟩⬛🟩\n...
3,Dad,2022-03-26 00:42:45,Wordle 280 3/6\n\n⬜🟩🟩⬜⬜\n🟩🟩🟩⬜⬜\n🟩🟩🟩🟩🟩
8,Michael,2022-03-25 19:05:29,Wordle 279 4/6\n\n🟨🟩⬛⬛⬛\n⬛🟩⬛🟨⬛\n🟨🟩🟩⬛🟨\n🟩🟩🟩🟩🟩\n...
9,Dad,2022-03-25 01:05:31,Wordle 279 3/6\n\n⬜⬜⬜⬜⬜\n🟨🟨⬜🟨⬜\n🟩🟩🟩🟩🟩
10,Danielle,2022-03-25 00:20:43,Wordle 279 4/6\n\n🟨🟩⬛⬛⬛\n⬛🟩🟨⬛⬛\n⬛🟩⬛🟨🟨\n🟩🟩🟩🟩🟩
...,...,...,...
574,Michael,2022-02-05 10:31:55,Wordle 231 4/6\n\n🟨⬛🟨⬛⬛\n🟩🟩⬛⬛⬛\n🟩🟩🟩⬛🟨\n🟩🟩🟩🟩🟩
576,Dad,2022-02-05 01:08:40,￼Wordle 231 3/6\n\n3 for me \n\n⬜🟨⬜🟨⬜\n🟨🟩🟩🟨🟩\n...
577,Michael,2022-02-04 08:41:31,Wordle 230 5/6\n\n🟨🟨⬛⬛⬛\n🟨🟩⬛⬛🟨\n⬛🟩🟩⬛🟨\n🟩🟩🟩🟩⬛\n...
578,Danielle,2022-02-04 08:39:54,Wordle 230 4/6\n\n⬛🟨⬛⬛🟨\n⬛🟨🟨🟨⬛\n🟩🟨🟨🟨🟩\n🟩🟩🟩🟩🟩


In [111]:
# creating a game number and score coluns

for rows in df:
    df['gamenumber'] = df.message[:6]

/Users/michaelharnett/opt/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [115]:
df = df.drop(columns=['gamenumber'])

In [116]:
df


,player,date,message
1,Dad,2022-03-26 00:50:25,😊
2,Danielle,2022-03-26 00:50:01,Wordle 280 5/6\n\n⬛⬛🟨⬛⬛\n⬛⬛🟩⬛🟨\n🟩⬛🟩⬛⬛\n🟩⬛🟩⬛🟩\n...
3,Dad,2022-03-26 00:42:45,Wordle 280 3/6\n\n⬜🟩🟩⬜⬜\n🟩🟩🟩⬜⬜\n🟩🟩🟩🟩🟩
4,Dad,2022-03-25 19:28:14,Thank you
5,Michael,2022-03-25 19:28:01,Shorthand for department
...,...,...,...
584,Michael,2022-02-03 19:12:58,I got wordle in 2 today 😏
585,Michael,2021-07-20 13:36:50,￼
586,Dad,2021-07-20 13:22:56,Thank you😊
587,Danielle,2021-07-20 13:22:22,Yes it’s my email and password is Lucky95


In [122]:
df.iloc[2]['message'][11:15]

'3/6\n'

In [123]:

## applying the funtion did not work since there was nothing keeping it at row level the whoe 
## df had the data from the last entry 

# def game_info_grabber (message):
#     '''
#     going to make a mini function to seperate the messages. I am going to make one colum
#     just the first 6 charachters. all messages I want should start with 'Wordle' this 
#     will allow another layer of filtering, making sure I am not capturing messages that
#     we were just using the word 'wordle' in
    
#     next will be game number,
    
#     then score.
#     '''
#     df['wordle_check'] = message[:6]
#     df['game_num'] = message[7:10]
#     df['score'] = message[11:14]    

In [124]:
df.message.apply(game_info_grabber)

1      None
2      None
3      None
4      None
5      None
       ... 
584    None
585    None
586    None
587    None
588    None
Name: message, Length: 585, dtype: object

In [129]:
df = df.drop(columns = ['wordle_check','game_num','score'])

In [127]:
df.message[:6]

1                                                    😊
2    Wordle 280 5/6\n\n⬛⬛🟨⬛⬛\n⬛⬛🟩⬛🟨\n🟩⬛🟩⬛⬛\n🟩⬛🟩⬛🟩\n...
3                Wordle 280 3/6\n\n⬜🟩🟩⬜⬜\n🟩🟩🟩⬜⬜\n🟩🟩🟩🟩🟩
4                                            Thank you
5                            Shorthand for department 
6                                                    ￼
Name: message, dtype: object

In [133]:
for rows in df:
    print(rows)

player
date
message


In [27]:
df['wordle_check'] = ''
df['game_num'] = ''
df['score'] = ''

In [40]:
## the loop below was breaking at index 540...I had removed nulls and
## didn't reset the index.
df = df.dropna()
df = df.reset_index()
df =df.drop(columns = ['index'])

In [41]:
df

,player,date,message,wordle_check,game_num,score
0,Chris,2022-03-26 09:10:04,Wordle 280 4/6\n\n⬛️🟨⬛️⬛️⬛️\n⬛️⬛️🟨⬛️⬛️\n⬛️🟨⬛️🟨...,,,
1,Dad,2022-03-26 00:50:25,😊,,,
2,Danielle,2022-03-26 00:50:01,Wordle 280 5/6\n\n⬛⬛🟨⬛⬛\n⬛⬛🟩⬛🟨\n🟩⬛🟩⬛⬛\n🟩⬛🟩⬛🟩\n...,,,
3,Dad,2022-03-26 00:42:45,Wordle 280 3/6\n\n⬜🟩🟩⬜⬜\n🟩🟩🟩⬜⬜\n🟩🟩🟩🟩🟩,,,
4,Dad,2022-03-25 19:28:14,Thank you,,,
...,...,...,...,...,...,...
581,Michael,2022-02-03 19:12:58,I got wordle in 2 today 😏,,,
582,Michael,2021-07-20 13:36:50,￼,,,
583,Dad,2021-07-20 13:22:56,Thank you😊,,,
584,Danielle,2021-07-20 13:22:22,Yes it’s my email and password is Lucky95,,,


In [42]:
for i in range(1,len(df)):
    df.loc[i,'wordle_check'] = df.loc[i,'message'][:6]
    df.loc[i,'game_num'] = df.loc[i,'message'][7:10]
    df.loc[i,'score'] = df.loc[i,'message'][11:14]
    

    

In [43]:
df[535:545]

,player,date,message,wordle_check,game_num,score
535,Michael,2022-02-12 22:12:47,I’m going to create a spreadsheet tomorrow to ...,I’m go,ng,o c
536,Chris,2022-02-12 22:08:39,Wordle 238 5/6\n\n⬛️🟨⬛️⬛️🟨\n🟨⬛️⬛️🟨🟨\n🟨⬛️🟨⬛️⬛️\...,Wordle,238,5/6
537,Dad,2022-02-12 01:04:00,Wordle 238 5/6\n\n⬜⬜⬜🟩⬜\n⬜⬜🟨🟩⬜\n⬜🟨⬜🟩⬜\n🟨⬜⬜🟩🟨\n...,Wordle,238,5/6
538,Danielle,2022-02-12 00:25:25,Wordle 238 3/6\n\n🟨🟨⬛⬛⬛\n⬛🟨🟨🟨⬛\n🟩🟩🟩🟩🟩,Wordle,238,3/6
539,Michael,2022-02-12 00:22:27,Wordle 238 4/6\n\n⬛🟩🟨🟨⬛\n⬛🟩⬛🟨🟨\n🟨🟩🟩⬛🟨\n🟩🟩🟩🟩🟩,Wordle,238,4/6
540,Chris,2022-02-11 19:58:32,Ello,Ello,,
541,Michael,2022-02-11 19:55:50,Welcome bruv,Welcom,br,v
542,Danielle,2022-02-11 12:07:55,Wordle 237 3/6\n\n⬛⬛🟨⬛🟨\n⬛🟨⬛🟩⬛\n🟩🟩🟩🟩🟩,Wordle,237,3/6
543,Michael,2022-02-11 06:58:51,Wordle 237 5/6\n\n⬛🟨⬛🟨⬛\n🟨⬛⬛🟩⬛\n🟨⬛⬛🟩🟩\n⬛🟨🟨🟩🟩\n...,Wordle,237,5/6
544,Dad,2022-02-11 01:32:41,Wordle 237 3/6\n\n⬜🟨⬜⬜⬜\n⬜⬜⬜⬜🟨\n🟩🟩🟩🟩🟩,Wordle,237,3/6


In [44]:
wordledf = df[df.wordle_check == 'Wordle']
wordledf

,player,date,message,wordle_check,game_num,score
2,Danielle,2022-03-26 00:50:01,Wordle 280 5/6\n\n⬛⬛🟨⬛⬛\n⬛⬛🟩⬛🟨\n🟩⬛🟩⬛⬛\n🟩⬛🟩⬛🟩\n...,Wordle,280,5/6
3,Dad,2022-03-26 00:42:45,Wordle 280 3/6\n\n⬜🟩🟩⬜⬜\n🟩🟩🟩⬜⬜\n🟩🟩🟩🟩🟩,Wordle,280,3/6
8,Michael,2022-03-25 19:05:29,Wordle 279 4/6\n\n🟨🟩⬛⬛⬛\n⬛🟩⬛🟨⬛\n🟨🟩🟩⬛🟨\n🟩🟩🟩🟩🟩\n...,Wordle,279,4/6
9,Dad,2022-03-25 01:05:31,Wordle 279 3/6\n\n⬜⬜⬜⬜⬜\n🟨🟨⬜🟨⬜\n🟩🟩🟩🟩🟩,Wordle,279,3/6
10,Danielle,2022-03-25 00:20:43,Wordle 279 4/6\n\n🟨🟩⬛⬛⬛\n⬛🟩🟨⬛⬛\n⬛🟩⬛🟨🟨\n🟩🟩🟩🟩🟩,Wordle,279,4/6
...,...,...,...,...,...,...
570,Danielle,2022-02-05 11:54:27,Wordle 231 4/6\n\n🟨⬛🟨⬛⬛\n⬛🟨⬛⬛🟩\n🟩⬛🟩⬛🟩\n🟩🟩🟩🟩🟩,Wordle,231,4/6
571,Michael,2022-02-05 10:31:55,Wordle 231 4/6\n\n🟨⬛🟨⬛⬛\n🟩🟩⬛⬛⬛\n🟩🟩🟩⬛🟨\n🟩🟩🟩🟩🟩,Wordle,231,4/6
574,Michael,2022-02-04 08:41:31,Wordle 230 5/6\n\n🟨🟨⬛⬛⬛\n🟨🟩⬛⬛🟨\n⬛🟩🟩⬛🟨\n🟩🟩🟩🟩⬛\n...,Wordle,230,5/6
575,Danielle,2022-02-04 08:39:54,Wordle 230 4/6\n\n⬛🟨⬛⬛🟨\n⬛🟨🟨🟨⬛\n🟩🟨🟨🟨🟩\n🟩🟩🟩🟩🟩,Wordle,230,4/6


In [212]:
# INTERESTING 
'''
was curious to see if python would consider this the same character. it does not! Maybe I can implement total yellows too,
like correctly guessed letters
'''


'🟨' == '🟩'

False

In [54]:
#wordledf = wordledf.reset_index()
wordledf = wordledf.drop(columns = ['index'])

In [61]:
### changing the score, to just the actual tires used, since they're all out of 6 guesses.\

for i in range(len(wordledf)):
    wordledf.loc[i, 'score'] = wordledf.loc[i, 'score'][0]

In [45]:
#dropping mesage column, imprtant datahas been extracted.

#also date column? date is not as important, if we're matching on game number, can be undone!

######ALTHOUGH super long term, we can maybe  

wordledf = wordledf.drop(columns = ['message', 'date'])
wordledf = wordledf.drop(columns = ['wordle_check'])

In [66]:
wordledf # TADA! this should be everything needed to start modeling really

,player,game_num,score
0,Danielle,280.0,5
1,Dad,280.0,3
2,Michael,279.0,4
3,Dad,279.0,3
4,Danielle,279.0,4
...,...,...,...
177,Danielle,231.0,4
178,Michael,231.0,4
179,Michael,230.0,5
180,Danielle,230.0,4


# visualizations and extrapolations


In [48]:
MichaelDF = wordledf[wordledf.player == 'Michael']
MichaelDF

,player,game_num,score


In [68]:
wordledf.player == 'Michael'


### thought this was a data type mismatch issue, BUTTTT it wound up being an extra space
### after I typed Michael the first time....fixed thtoughout the module.

0      False
1      False
2      False
3      False
4      False
       ...  
177    False
178    False
179    False
180    False
181    False
Name: player, Length: 182, dtype: bool

###### the fuck


In [63]:
wordledf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   player    182 non-null    object
 1   game_num  182 non-null    object
 2   score     182 non-null    object
dtypes: object(3)
memory usage: 4.4+ KB


In [74]:
wordledf.player = wordledf.player.astype('str')
wordledf.game_num = wordledf.game_num.astype('int')
wordledf.score = wordledf.score.astype('str')

In [69]:
wordledf.player.value_counts()

Danielle    50
Dad         48
Michael     44
Chris       40
Name: player, dtype: int64

In [91]:
michaeldf = wordledf[wordledf.player == 'Michael']

danielledf = wordledf[wordledf.player == 'Danielle']

chrisdf = wordledf[wordledf.player == 'Chris']

daddf = wordledf[wordledf.player == 'Dad']


    player  game_num score
2  Michael       279     4
7  Michael       278     5      player  game_num score
0  Danielle       280     5
4  Danielle       279     4   player  game_num score
5  Chris       279     2
6  Chris       278     3   player  game_num score
1    Dad       280     3
3    Dad       279     3


In [92]:
wordledf['LOSS'] = ''

In [101]:


#wordledf[wordledf.score == 'X']
#wordledf.loc[wordledf.score == 'X','LOSS'] = 1

,player,game_num,score,LOSS
11,Michael,277,X,1
50,Michael,265,X,1
63,Michael,262,X,1
106,Michael,251,X,1
140,Danielle,243,X,1


In [95]:
wordledf.score.value_counts()

4    56
5    54
3    43
6    15
2     8
X     5
x     1
Name: score, dtype: int64